In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict
import os

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 40,
  CHOOSE = 6,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5, 6],
  MAX_B_VALUES = [35, 36, 37, 38, 39, 40]
)


In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=42'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_b5 = None
post_str_blast = None
for row in rows:
  for col_index in [2, 3]:
    cols = [data.text.strip() for data in row.find_all('td')]
    if (cols[col_index] == ''):
      continue

    # cols = [data.text.strip() for data in row.find_all('td')]
    
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)
    cols.append(post_str_b1)
    cols.append(post_str_b2)
    cols.append(post_str_b3)
    cols.append(post_str_b4)
    cols.append(post_str_b5)
    cols.append(post_str_blast)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)

    post_str_b1 = str_combo[0]
    post_str_b2 = str_combo[1]
    post_str_b3 = str_combo[2]
    post_str_b4 = str_combo[3]
    post_str_b5 = str_combo[4]
    post_str_blast = str_combo[-1]



# lst = []
# for row in rows:
  
  
#   for col_index in [2, 3]:
#     cols = [data.text.strip() for data in row.find_all('td')]
#     if (cols[col_index] == ''):
#       continue
  
#     id, bolillas_int = formalize_data(bolillas_str=cols[col_index])
#     cols.append(id)
#     cols.append(bolillas_int)
#     cols.extend([b for b in bolillas_int])
    
#     is_rare, reason = rr.is_rare_combination(bolillas_int, id)
#     cols.append(is_rare)
#     cols.append(reason)
#     left_scores = [round(b, 4) for b in rr.calculate_left_scores(bolillas_int)]

#     right_scores = [round(b, 4) for b in rr.calculate_right_scores(bolillas_int)]
#     lr_scores = [round(left + right,4) for left, right in zip(left_scores, right_scores)]
#     row_score = round(sum(lr_scores),4)

#     shared_values = list(set(left_scores[0:5]) & set(right_scores[1:6]))
#     shared_values = [round(s, 4) for s in shared_values]
#     cols.extend([score for score in left_scores])
#     cols.extend([score for score in right_scores])
#     # cols.extend(lr_scores)
#     cols.append(shared_values)
#     cols.append(len(shared_values))
#     cols.append(row_score)
#     cols.append(rr.jumps_map(bolillas_int))
#     cols.append(rr.unijump(bolillas_int))
    
#     lst.append(cols)

with open('./kbl_cache_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 6),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=5282


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""",null,null,null,null,null,null,false,0,"""02_01_18_12_06""",64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""04_06_03_22_01""",-89
"""17/04/2025""","""1796""","""12 06 38 15 37 02""","""21 11 28 24 32 20""","""""","""""","""020612153738""","[2, 6, … 38]",2,6,12,15,37,38,"""02""","""06""","""12""","""15""","""37""","""38""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""04_06_03_22_01""",-89
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""03_05_08_13_03""",-18
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""03_05_08_13_03""",-18
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""07_01_08_07_06""",29
"""12/04/2025""","""1794""","""10 32 26 03 19 11""","""39 07 28 21 05 12""","""""","""""","""031011192632""","[3, 10, … 32]",3,10,11,19,26,32,"""03""","""10""","""11""","""19""","""26""","""32""","""03""","""10""","""11""","""19""","""26""","""32""",false,0,"""07_01_08_07_06""",29
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""03""","""10""","""11""","""19""","""26""","""32""",true,8,"""01_16_02_02_14""",-45


In [5]:
dfRows.item(0, 'b2')

3

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  

  print(f'df_10_last FOR {col} = {test_value}')
  last_results = list(set(df_10_last[post_col][0:3]))
  print(f'Last 3 results {last_results=}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map, last_results

In [7]:
possible_b1, exclude_jump_maps_b1, last_post_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='01'
df_viz FOR str_b1 = 01


post_str_b1,count,percent
str,u32,f64
"""01""",330,42.253521
"""03""",72,9.21895
"""02""",64,8.194622
"""04""",62,7.93854
"""05""",42,5.377721
…,…,…
"""28""",1,0.128041
"""17""",1,0.128041
"""23""",1,0.128041


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 01
Last 3 results last_results=['03', '01']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""03""","""10""","""11""","""19""","""26""","""32""",true,8,"""01_16_02_02_14""",-45
"""10/04/2025""","""1793""","""01 36 20 18 22 02""","""33 17 29 39 30 36""","""""","""""","""010218202236""","[1, 2, … 36]",1,2,18,20,22,36,"""01""","""02""","""18""","""20""","""22""","""36""","""01""","""02""","""18""","""20""","""22""","""36""",true,8,"""01_16_02_02_14""",-45
"""22/02/2025""","""1773""","""25 15 26 01 07 18""","""13 38 06 37 18 20""","""""","""""","""010715182526""","[1, 7, … 26]",1,7,15,18,25,26,"""01""","""07""","""15""","""18""","""25""","""26""","""03""","""06""","""26""","""31""","""32""","""40""",true,16,"""06_08_03_07_01""",-35
"""22/02/2025""","""1773""","""25 15 26 01 07 18""","""13 38 06 37 18 20""","""""","""""","""010715182526""","[1, 7, … 26]",1,7,15,18,25,26,"""01""","""07""","""15""","""18""","""25""","""26""","""01""","""07""","""15""","""18""","""25""","""26""",true,16,"""06_08_03_07_01""",-35
"""28/01/2025""","""1762""","""39 24 01 06 14 18""","""20 13 22 37 07 11""","""""","""""","""010614182439""","[1, 6, … 39]",1,6,14,18,24,39,"""01""","""06""","""14""","""18""","""24""","""39""","""04""","""20""","""31""","""33""","""36""","""38""",false,0,"""05_08_04_06_15""",-12
"""28/01/2025""","""1762""","""39 24 01 06 14 18""","""20 13 22 37 07 11""","""""","""""","""010614182439""","[1, 6, … 39]",1,6,14,18,24,39,"""01""","""06""","""14""","""18""","""24""","""39""","""01""","""06""","""14""","""18""","""24""","""39""",false,0,"""05_08_04_06_15""",-12
"""02/01/2025""","""1751""","""10 34 24 16 35 01""","""11 03 18 29 15 28""","""""","""""","""011016243435""","[1, 10, … 35]",1,10,16,24,34,35,"""01""","""10""","""16""","""24""","""34""","""35""","""08""","""12""","""19""","""27""","""37""","""39""",false,0,"""09_06_08_10_01""",-6
"""02/01/2025""","""1751""","""10 34 24 16 35 01""","""11 03 18 29 15 28""","""""","""""","""011016243435""","[1, 10, … 35]",1,10,16,24,34,35,"""01""","""10""","""16""","""24""","""34""","""35""","""01""","""10""","""16""","""24""","""34""","""35""",false,0,"""09_06_08_10_01""",-6


df_jumps_map FOR str_b1 = 01


jumps_map,count
str,u32
"""08_09_09_08_01""",2
"""09_04_13_03_09""",2
"""04_07_02_23_01""",2
"""11_06_11_01_10""",2
"""06_11_11_03_04""",2
…,…
"""06_09_16_01_06""",1
"""07_02_13_02_08""",1
"""01_14_02_17_02""",1


df_unijump FOR str_b1 = 01


unijump,count
i64,u32
-22,16
-51,13
-21,13
-17,11
-11,11
…,…
74,1
-75,1
106,1


In [8]:
possible_b2, exclude_jump_maps_b2, last_post_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='03'
df_viz FOR str_b2 = 03


post_str_b2,count,percent
str,u32,f64
"""03""",64,35.75419
"""10""",11,6.145251
"""09""",10,5.586592
"""08""",10,5.586592
"""04""",9,5.027933
…,…,…
"""18""",2,1.117318
"""19""",2,1.117318
"""11""",2,1.117318


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 03
Last 3 results last_results=['03', '08']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""18/01/2025""","""1758""","""28 02 18 03 16 26""","""34 36 24 38 01 20""","""""","""""","""020316182628""","[2, 3, … 28]",2,3,16,18,26,28,"""02""","""03""","""16""","""18""","""26""","""28""","""06""","""08""","""16""","""19""","""20""","""38""",false,0,"""01_13_02_08_02""",-69
"""18/01/2025""","""1758""","""28 02 18 03 16 26""","""34 36 24 38 01 20""","""""","""""","""020316182628""","[2, 3, … 28]",2,3,16,18,26,28,"""02""","""03""","""16""","""18""","""26""","""28""","""02""","""03""","""16""","""18""","""26""","""28""",false,0,"""01_13_02_08_02""",-69
"""07/12/2024""","""1740""","""18 26 01 16 03 23""","""22 10 39 23 06 25""","""""","""""","""010316182326""","[1, 3, … 26]",1,3,16,18,23,26,"""01""","""03""","""16""","""18""","""23""","""26""","""01""","""12""","""14""","""19""","""35""","""38""",false,0,"""02_13_02_05_03""",-55
"""07/12/2024""","""1740""","""18 26 01 16 03 23""","""22 10 39 23 06 25""","""""","""""","""010316182326""","[1, 3, … 26]",1,3,16,18,23,26,"""01""","""03""","""16""","""18""","""23""","""26""","""01""","""03""","""16""","""18""","""23""","""26""",false,0,"""02_13_02_05_03""",-55
"""18/05/2024""","""1653""","""35 03 01 27 14 05""","""30 05 29 19 39 13""","""""","""""","""010305142735""","[1, 3, … 35]",1,3,5,14,27,35,"""01""","""03""","""05""","""14""","""27""","""35""","""01""","""04""","""05""","""16""","""18""","""37""",true,8,"""02_02_09_13_08""",4
"""18/05/2024""","""1653""","""35 03 01 27 14 05""","""30 05 29 19 39 13""","""""","""""","""010305142735""","[1, 3, … 35]",1,3,5,14,27,35,"""01""","""03""","""05""","""14""","""27""","""35""","""01""","""03""","""05""","""14""","""27""","""35""",true,8,"""02_02_09_13_08""",4
"""25/04/2024""","""1643""","""18 01 03 21 33 28""","""35 27 19 32 34 39""","""""","""""","""010318212833""","[1, 3, … 33]",1,3,18,21,28,33,"""01""","""03""","""18""","""21""","""28""","""33""","""09""","""10""","""12""","""15""","""17""","""40""",false,0,"""02_15_03_07_05""",-63
"""25/04/2024""","""1643""","""18 01 03 21 33 28""","""35 27 19 32 34 39""","""""","""""","""010318212833""","[1, 3, … 33]",1,3,18,21,28,33,"""01""","""03""","""18""","""21""","""28""","""33""","""01""","""03""","""18""","""21""","""28""","""33""",false,0,"""02_15_03_07_05""",-63


df_jumps_map FOR str_b2 = 03


jumps_map,count
str,u32
"""02_07_03_04_04""",2
"""01_23_02_03_09""",2
"""02_11_06_12_05""",2
"""02_15_03_07_05""",2
"""01_14_09_04_06""",2
…,…
"""02_04_02_07_03""",1
"""02_01_18_12_06""",1
"""01_08_15_01_06""",1


df_unijump FOR str_b2 = 03


unijump,count
i64,u32
-17,5
-46,5
14,4
4,4
-3,4
…,…
-33,1
-7,1
-107,1


In [9]:
possible_b3, exclude_jump_maps_b3, last_post_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='04'
df_viz FOR str_b3 = 04


post_str_b3,count,percent
str,u32,f64
"""04""",9,39.130435
"""14""",2,8.695652
"""17""",2,8.695652
"""29""",2,8.695652
"""30""",1,4.347826
…,…,…
"""06""",1,4.347826
"""15""",1,4.347826
"""18""",1,4.347826


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 04
Last 3 results last_results=['04', '29']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""20/04/2021""","""1171""","""04 40 11 02 01 21""","""20 29 30 02 34 38""","""""","""""","""010204112140""","[1, 2, … 40]",1,2,4,11,21,40,"""01""","""02""","""04""","""11""","""21""","""40""","""06""","""24""","""29""","""31""","""34""","""35""",true,16,"""01_02_07_10_19""",14
"""20/04/2021""","""1171""","""04 40 11 02 01 21""","""20 29 30 02 34 38""","""""","""""","""010204112140""","[1, 2, … 40]",1,2,4,11,21,40,"""01""","""02""","""04""","""11""","""21""","""40""","""01""","""02""","""04""","""11""","""21""","""40""",true,16,"""01_02_07_10_19""",14
"""19/09/2020""","""1080""","""31 04 01 02 06 11""","""07 27 19 10 02 03""","""""","""""","""010204061131""","[1, 2, … 31]",1,2,4,6,11,31,"""01""","""02""","""04""","""06""","""11""","""31""","""03""","""07""","""18""","""23""","""27""","""32""",true,8,"""01_02_02_05_20""",5
"""19/09/2020""","""1080""","""31 04 01 02 06 11""","""07 27 19 10 02 03""","""""","""""","""010204061131""","[1, 2, … 31]",1,2,4,6,11,31,"""01""","""02""","""04""","""06""","""11""","""31""","""01""","""02""","""04""","""06""","""11""","""31""",true,8,"""01_02_02_05_20""",5
"""14/07/2020""","""1051""","""01 14 09 03 04 30""","""40 37 19 15 26 20""","""""","""""","""010304091430""","[1, 3, … 30]",1,3,4,9,14,30,"""01""","""03""","""04""","""09""","""14""","""30""","""03""","""05""","""17""","""18""","""20""","""21""",true,24,"""02_01_05_05_16""",24
"""14/07/2020""","""1051""","""01 14 09 03 04 30""","""40 37 19 15 26 20""","""""","""""","""010304091430""","[1, 3, … 30]",1,3,4,9,14,30,"""01""","""03""","""04""","""09""","""14""","""30""","""01""","""03""","""04""","""09""","""14""","""30""",true,24,"""02_01_05_05_16""",24
"""12/01/2019""","""0849""","""13 14 17 04 03 01""","""10 26 22 09 08 16""","""""","""""","""010304131417""","[1, 3, … 17]",1,3,4,13,14,17,"""01""","""03""","""04""","""13""","""14""","""17""","""11""","""25""","""29""","""34""","""38""","""40""",true,16,"""02_01_09_01_03""",51
"""12/01/2019""","""0849""","""13 14 17 04 03 01""","""10 26 22 09 08 16""","""""","""""","""010304131417""","[1, 3, … 17]",1,3,4,13,14,17,"""01""","""03""","""04""","""13""","""14""","""17""","""01""","""03""","""04""","""13""","""14""","""17""",true,16,"""02_01_09_01_03""",51


df_jumps_map FOR str_b3 = 04


jumps_map,count
str,u32
"""02_01_09_01_03""",2
"""01_02_02_28_03""",2
"""02_01_03_26_01""",2
"""02_01_03_01_16""",2
"""01_02_07_10_19""",2
…,…
"""01_01_16_03_11""",1
"""02_01_18_12_06""",1
"""01_02_09_08_08""",1


df_unijump FOR str_b3 = 04


unijump,count
i64,u32
5,2
-87,2
-104,2
126,2
51,2
…,…
-44,1
-66,1
-57,1


In [10]:
possible_b4, exclude_jump_maps_b4, last_post_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='22'
df_viz FOR str_b4 = 22


post_str_b4,count,percent
str,u32,f64
"""22""",118,38.562092
"""26""",15,4.901961
"""25""",14,4.575163
"""27""",14,4.575163
"""19""",12,3.921569
…,…,…
"""10""",2,0.653595
"""17""",2,0.653595
"""08""",2,0.653595


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 22
Last 3 results last_results=['13', '22']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""18/03/2025""","""1783""","""39 16 22 21 30 12""","""33 39 24 05 20 40""","""""","""""","""121621223039""","[12, 16, … 39]",12,16,21,22,30,39,"""12""","""16""","""21""","""22""","""30""","""39""","""03""","""07""","""11""","""13""","""15""","""35""",false,0,"""04_05_01_08_09""",-33
"""18/03/2025""","""1783""","""39 16 22 21 30 12""","""33 39 24 05 20 40""","""""","""""","""121621223039""","[12, 16, … 39]",12,16,21,22,30,39,"""12""","""16""","""21""","""22""","""30""","""39""","""12""","""16""","""21""","""22""","""30""","""39""",false,0,"""04_05_01_08_09""",-33
"""27/02/2025""","""1775""","""08 03 29 25 07 22""","""09 10 08 16 24 07""","""""","""""","""030708222529""","[3, 7, … 29]",3,7,8,22,25,29,"""03""","""07""","""08""","""22""","""25""","""29""","""02""","""14""","""22""","""34""","""38""","""40""",true,16,"""04_01_14_03_04""",76
"""27/02/2025""","""1775""","""08 03 29 25 07 22""","""09 10 08 16 24 07""","""""","""""","""030708222529""","[3, 7, … 29]",3,7,8,22,25,29,"""03""","""07""","""08""","""22""","""25""","""29""","""03""","""07""","""08""","""22""","""25""","""29""",true,16,"""04_01_14_03_04""",76
"""18/02/2025""","""1771""","""39 04 17 07 28 22""","""16 29 36 22 07 11""","""""","""""","""040717222839""","[4, 7, … 39]",4,7,17,22,28,39,"""04""","""07""","""17""","""22""","""28""","""39""","""03""","""05""","""09""","""21""","""27""","""32""",false,0,"""03_10_05_06_11""",-20
"""18/02/2025""","""1771""","""39 04 17 07 28 22""","""16 29 36 22 07 11""","""""","""""","""040717222839""","[4, 7, … 39]",4,7,17,22,28,39,"""04""","""07""","""17""","""22""","""28""","""39""","""04""","""07""","""17""","""22""","""28""","""39""",false,0,"""03_10_05_06_11""",-20
"""08/02/2025""","""1767""","""34 22 16 17 21 29""","""38 05 33 35 07 06""","""""","""""","""161721222934""","[16, 17, … 34]",16,17,21,22,29,34,"""16""","""17""","""21""","""22""","""29""","""34""","""11""","""12""","""15""","""20""","""30""","""32""",false,0,"""01_04_01_07_05""",-32
"""08/02/2025""","""1767""","""34 22 16 17 21 29""","""38 05 33 35 07 06""","""""","""""","""161721222934""","[16, 17, … 34]",16,17,21,22,29,34,"""16""","""17""","""21""","""22""","""29""","""34""","""16""","""17""","""21""","""22""","""29""","""34""",false,0,"""01_04_01_07_05""",-32


df_jumps_map FOR str_b4 = 22


jumps_map,count
str,u32
"""06_06_05_04_12""",2
"""04_06_11_01_10""",2
"""01_03_09_09_05""",2
"""04_06_01_07_06""",2
"""03_04_02_01_14""",2
…,…
"""07_05_02_09_01""",1
"""03_07_11_04_09""",1
"""07_01_10_09_07""",1


df_unijump FOR str_b4 = 22


unijump,count
i64,u32
-7,11
1,8
9,8
-10,8
-37,8
…,…
55,1
-22,1
13,1


In [11]:
possible_b5, exclude_jump_maps_b5, last_post_b5 = plot_post_col(col='str_b5', post_col='post_str_b5')

Setting test_value='34'
df_viz FOR str_b5 = 34


post_str_b5,count,percent
str,u32,f64
"""34""",116,34.319527
"""30""",21,6.213018
"""27""",16,4.733728
"""33""",16,4.733728
"""31""",14,4.142012
…,…,…
"""13""",2,0.591716
"""15""",2,0.591716
"""11""",1,0.295858


:Overlay
   .Bars.I    :Bars   [post_str_b5]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 34
Last 3 results last_results=['34', '37']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""02""","""06""","""12""","""15""","""37""","""38""",false,0,"""03_05_08_13_03""",-18
"""15/04/2025""","""1795""","""05 21 37 13 08 34""","""38 07 03 09 04 27""","""""","""""","""050813213437""","[5, 8, … 37]",5,8,13,21,34,37,"""05""","""08""","""13""","""21""","""34""","""37""","""05""","""08""","""13""","""21""","""34""","""37""",false,0,"""03_05_08_13_03""",-18
"""03/04/2025""","""1790""","""16 17 32 34 39 13""","""26 31 36 34 28 08""","""""","""""","""131617323439""","[13, 16, … 39]",13,16,17,32,34,39,"""13""","""16""","""17""","""32""","""34""","""39""","""13""","""18""","""22""","""23""","""30""","""39""",false,0,"""03_01_15_02_05""",86
"""03/04/2025""","""1790""","""16 17 32 34 39 13""","""26 31 36 34 28 08""","""""","""""","""131617323439""","[13, 16, … 39]",13,16,17,32,34,39,"""13""","""16""","""17""","""32""","""34""","""39""","""13""","""16""","""17""","""32""","""34""","""39""",false,0,"""03_01_15_02_05""",86
"""01/04/2025""","""1789""","""32 34 28 36 22 14""","""22 27 17 14 08 15""","""""","""""","""142228323436""","[14, 22, … 36]",14,22,28,32,34,36,"""14""","""22""","""28""","""32""","""34""","""36""","""13""","""16""","""17""","""32""","""34""","""39""",true,12,"""08_06_04_02_02""",2
"""01/04/2025""","""1789""","""32 34 28 36 22 14""","""22 27 17 14 08 15""","""""","""""","""142228323436""","[14, 22, … 36]",14,22,28,32,34,36,"""14""","""22""","""28""","""32""","""34""","""36""","""14""","""22""","""28""","""32""","""34""","""36""",true,12,"""08_06_04_02_02""",2
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""07""","""19""","""21""","""28""","""40""",false,0,"""04_02_03_19_05""",-57
"""11/03/2025""","""1780""","""15 12 34 10 06 39""","""10 27 16 07 02 40""","""""","""""","""061012153439""","[6, 10, … 39]",6,10,12,15,34,39,"""06""","""10""","""12""","""15""","""34""","""39""","""06""","""10""","""12""","""15""","""34""","""39""",false,0,"""04_02_03_19_05""",-57


df_jumps_map FOR str_b5 = 34


jumps_map,count
str,u32
"""01_12_08_02_05""",2
"""07_09_13_04_05""",2
"""05_09_11_02_05""",2
"""20_02_03_04_05""",2
"""01_10_12_10_04""",2
…,…
"""10_02_11_06_02""",1
"""16_10_04_01_03""",1
"""12_03_06_04_01""",1


df_unijump FOR str_b5 = 34


unijump,count
i64,u32
-1,8
30,6
-26,6
-17,6
-2,5
…,…
40,1
45,1
-94,1


In [12]:
possible_blast, exclude_jump_maps_blast, last_post_blast = plot_post_col(col='str_b6', post_col='post_str_blast')

Setting test_value='40'
df_viz FOR str_b6 = 40


post_str_blast,count,percent
str,u32,f64
"""40""",362,43.825666
"""39""",70,8.474576
"""38""",57,6.900726
"""37""",50,6.053269
"""35""",47,5.690073
…,…,…
"""19""",1,0.121065
"""18""",1,0.121065
"""13""",1,0.121065


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b6 = 40
Last 3 results last_results=['36', '40']


fecha,sorteo,bolillas regulares,bolillas chauchamba,cc1,cc2,id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_b5,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,str,str,str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""19/04/2025""","""1797""","""40 34 04 03 01 22""","""06 02 22 09 04 21""","""""","""""","""010304223440""","[1, 3, … 40]",1,3,4,22,34,40,"""01""","""03""","""04""","""22""","""34""","""40""","""01""","""03""","""04""","""22""","""34""","""40""",false,0,"""02_01_18_12_06""",64
"""08/04/2025""","""1792""","""17 39 38 04 40 28""","""23 34 17 08 24 01""","""""","""""","""041728383940""","[4, 17, … 40]",4,17,28,38,39,40,"""04""","""17""","""28""","""38""","""39""","""40""","""01""","""02""","""18""","""20""","""22""","""36""",true,8,"""13_11_10_01_01""",26
"""08/04/2025""","""1792""","""17 39 38 04 40 28""","""23 34 17 08 24 01""","""""","""""","""041728383940""","[4, 17, … 40]",4,17,28,38,39,40,"""04""","""17""","""28""","""38""","""39""","""40""","""04""","""17""","""28""","""38""","""39""","""40""",true,8,"""13_11_10_01_01""",26
"""29/03/2025""","""1788""","""36 31 33 40 23 25""","""38 05 16 07 09 24""","""""","""""","""232531333640""","[23, 25, … 40]",23,25,31,33,36,40,"""23""","""25""","""31""","""33""","""36""","""40""","""14""","""22""","""28""","""32""","""34""","""36""",false,0,"""02_06_02_03_04""",-18
"""29/03/2025""","""1788""","""36 31 33 40 23 25""","""38 05 16 07 09 24""","""""","""""","""232531333640""","[23, 25, … 40]",23,25,31,33,36,40,"""23""","""25""","""31""","""33""","""36""","""40""","""23""","""25""","""31""","""33""","""36""","""40""",false,0,"""02_06_02_03_04""",-18
"""13/03/2025""","""1781""","""21 19 06 07 28 40""","""16 15 28 27 25 05""","""""","""""","""060719212840""","[6, 7, … 40]",6,7,19,21,28,40,"""06""","""07""","""19""","""21""","""28""","""40""","""03""","""09""","""10""","""19""","""28""","""39""",true,16,"""01_12_02_07_12""",-51
"""13/03/2025""","""1781""","""21 19 06 07 28 40""","""16 15 28 27 25 05""","""""","""""","""060719212840""","[6, 7, … 40]",6,7,19,21,28,40,"""06""","""07""","""19""","""21""","""28""","""40""","""06""","""07""","""19""","""21""","""28""","""40""",true,16,"""01_12_02_07_12""",-51
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""04""","""06""","""11""","""18""","""26""","""30""",true,4,"""12_08_12_04_02""",38
"""01/03/2025""","""1776""","""02 34 38 14 22 40""","""09 08 36 39 14 01""","""""","""""","""021422343840""","[2, 14, … 40]",2,14,22,34,38,40,"""02""","""14""","""22""","""34""","""38""","""40""","""02""","""14""","""22""","""34""","""38""","""40""",true,4,"""12_08_12_04_02""",38


df_jumps_map FOR str_b6 = 40


jumps_map,count
str,u32
"""06_07_03_10_05""",2
"""10_08_01_03_10""",2
"""01_06_05_14_01""",2
"""09_02_18_02_02""",2
"""04_02_05_06_07""",2
…,…
"""05_24_02_02_03""",1
"""03_01_13_07_02""",1
"""05_03_03_10_14""",1


df_unijump FOR str_b6 = 40


unijump,count
i64,u32
-17,18
-11,16
27,14
-51,14
4,13
…,…
-59,1
116,1
-63,1


In [13]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [14]:
parquet_file = './parquet_kbl_v3.parquet'

# check if parquet_file exists
if not os.path.exists(parquet_file):
  print(f'{parquet_file=} does not exists. creating')
  lst = []

  for row in combos:
    cols = []
    str_row = " ".join([str(r).zfill(2) for r in row])
    id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
    cols.append(id)
    cols.append(bolillas_int)
    cols.extend([b for b in bolillas_int])
    cols.extend(str_combo)

    is_rare, reason = rr.is_rare_combination(bolillas_int, id)
    cols.append(is_rare)
    cols.append(reason)

    cols.append(rr.jumps_map(bolillas_int))
    cols.append(rr.unijump(bolillas_int))
    
    lst.append(cols)


  with open('./gn_cahe_v3.json', 'w') as file: 
    json.dump(lst, file)

  
  # dfRows = pl.DataFrame(
  #   lst,
  #   schema=[
  #     'fecha', 'sorteo', 'bolillas regulares', 'bolillas chauchamba', 'cc1', 'cc2',
  #     'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
  #     'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
  #     'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_b5', 'post_str_blast',
  #     'is_rare_combination', 'reason',
  #     "jumps_map", "unijump"
  #   ],
  #   schema_overrides={
  #     'combo': pl.Array(pl.UInt8, 6),
  #     'b1': pl.UInt8,
  #     'b2': pl.UInt8,
  #     'b3': pl.UInt8,
  #     'b4': pl.UInt8,
  #     'b5': pl.UInt8,
  #     'b6': pl.UInt8,
  #     'reason': pl.UInt8
  #   },
  #   orient='row'
  # )

  df_Orows = pl.DataFrame(
    lst,
    schema=[
      # 'fecha', 'sorteo', 'bolillas',
      'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
      'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5', 'str_b6',
      'is_rare_combination', 'reason',
      "jumps_map", "unijump"
    ],
    schema_overrides={
      'combo': pl.Array(pl.UInt8, 6),
      'b1': pl.UInt8,
      'b2': pl.UInt8,
      'b3': pl.UInt8,
      'b4': pl.UInt8,
      'b5': pl.UInt8,
      'b6': pl.UInt8,
      'reason': pl.UInt8
    },
    orient='row'
  )

  df_Orows.write_parquet(parquet_file)
  TOTAL_LINES = len(df_Orows)
  print(f'{TOTAL_LINES=} loaded to {parquet_file=}')

  display(df_Orows.head(10))
else:
  df_Orows = pl.read_parquet(parquet_file)

In [15]:
# possible_b1, exclude_jump_maps_b1
df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  pl.col("str_b5").is_in(possible_b5),
  pl.col("str_b6").is_in(possible_blast),
  
  ~pl.col("str_b1").is_in(last_post_b1),
  ~pl.col("str_b2").is_in(last_post_b2),
  ~pl.col("str_b3").is_in(last_post_b3),
  ~pl.col("str_b4").is_in(last_post_b4),
  ~pl.col("str_b5").is_in(last_post_b5),
  ~pl.col("str_b6").is_in(last_post_blast),

  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_b5 + exclude_jump_maps_blast),
  pl.col("is_rare_combination") == False
)


display(df_possible)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""020414192634""","[2, 4, … 34]",2,4,14,19,26,34,"""02""","""04""","""14""","""19""","""26""","""34""",false,0,"""02_10_05_07_08""",-28
"""020414192635""","[2, 4, … 35]",2,4,14,19,26,35,"""02""","""04""","""14""","""19""","""26""","""35""",false,0,"""02_10_05_07_09""",-27
"""020414192637""","[2, 4, … 37]",2,4,14,19,26,37,"""02""","""04""","""14""","""19""","""26""","""37""",false,0,"""02_10_05_07_11""",-25
"""020414192638""","[2, 4, … 38]",2,4,14,19,26,38,"""02""","""04""","""14""","""19""","""26""","""38""",false,0,"""02_10_05_07_12""",-24
"""020414192639""","[2, 4, … 39]",2,4,14,19,26,39,"""02""","""04""","""14""","""19""","""26""","""39""",false,0,"""02_10_05_07_13""",-23
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""081317273139""","[8, 13, … 39]",8,13,17,27,31,39,"""08""","""13""","""17""","""27""","""31""","""39""",false,0,"""05_04_10_04_08""",41
"""081317273334""","[8, 13, … 34]",8,13,17,27,33,34,"""08""","""13""","""17""","""27""","""33""","""34""",false,0,"""05_04_10_06_01""",26
"""081317273335""","[8, 13, … 35]",8,13,17,27,33,35,"""08""","""13""","""17""","""27""","""33""","""35""",false,0,"""05_04_10_06_02""",27


In [16]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,b6,str_b1,str_b2,str_b3,str_b4,str_b5,str_b6,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 6]",u8,u8,u8,u8,u8,u8,str,str,str,str,str,str,bool,u8,str,i64
"""020414273334""","[2, 4, … 34]",2,4,14,27,33,34,"""02""","""04""","""14""","""27""","""33""","""34""",false,0,"""02_10_13_06_01""",17
